## Simulating melts of polymer chains

In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import gsd.hoomd
from hoomd.md import nlist

import unyt as u
import glob
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
import os

import flowermd
from flowermd.utils import get_target_box_mass_density
from flowermd.base import Simulation, Molecule
from flowermd.library import FF_from_file
from flowermd.base.system import Pack

from polymer_dictionary import bond_indices

# Rerun with the original seed first

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/pkg_resources/__init__.py:3146: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)



Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



In [2]:
# Updates atom names to fit with hoomd conventions
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

## Single molecule sim:

In [3]:
key_list = sorted(list(bond_indices.keys())) # Aligning dictionary and path to aid automation
path = os.getcwd() # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [4]:
mol2_path = os.getcwd() + "/mol2"
xml_path = os.getcwd() + "/xml"
system_file = '/home/jbieri/CME_lab/repos/pl-validation/mol2/PCPDTFBT_C4_BO_10mer.mol2'
ff_filepath = '/home/jbieri/CME_lab/repos/pl-validation/xml/PCPDTFBT_C4_BO.xml'

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.


In [16]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=1000, log_write_freq=1000, device=cpu, gsd_file_name='PCPDT_PT_eneHD_30mer.gsd',log_file_name="PCPDT_PT_eneHD_30mer.txt")

Initializing simulation state from a gsd.hoomd.Frame.


In [17]:
sim.run_NVT(n_steps=1e6, kT=5.0, tau_kt=1.0)
sim.flush_writers()

Step 1000 of 1000000; TPS: 442.02; ETA: 37.7 minutes
Step 2000 of 1000000; TPS: 371.83; ETA: 44.7 minutes
Step 3000 of 1000000; TPS: 429.6; ETA: 38.7 minutes
Step 4000 of 1000000; TPS: 468.93; ETA: 35.4 minutes
Step 5000 of 1000000; TPS: 492.29; ETA: 33.7 minutes
Step 6000 of 1000000; TPS: 512.72; ETA: 32.3 minutes
Step 7000 of 1000000; TPS: 528.56; ETA: 31.3 minutes
Step 8000 of 1000000; TPS: 542.04; ETA: 30.5 minutes
Step 9000 of 1000000; TPS: 552.68; ETA: 29.9 minutes
Step 10000 of 1000000; TPS: 560.93; ETA: 29.4 minutes
Step 11000 of 1000000; TPS: 569.29; ETA: 29.0 minutes
Step 12000 of 1000000; TPS: 574.05; ETA: 28.7 minutes
Step 13000 of 1000000; TPS: 579.4; ETA: 28.4 minutes
Step 14000 of 1000000; TPS: 583.76; ETA: 28.2 minutes
Step 15000 of 1000000; TPS: 585.78; ETA: 28.0 minutes
Step 16000 of 1000000; TPS: 584.64; ETA: 28.1 minutes
Step 17000 of 1000000; TPS: 584.97; ETA: 28.0 minutes
Step 18000 of 1000000; TPS: 588.15; ETA: 27.8 minutes
Step 19000 of 1000000; TPS: 591.45; ETA

In [24]:
sim.reference_values

{'energy': unyt_quantity(1.046, 'kJ/mol'),
 'length': unyt_quantity(0.35635949, 'nm'),
 'mass': unyt_quantity(32.06, 'amu')}

In [3]:
system_file = '/home/jbieri/CME_lab/repos/pl-validation/mol2/PCPDTFBT_C4_BO_10mer.mol2'
ff_filepath = '/home/jbieri/CME_lab/repos/pl-validation/xml/PCPDTFBT_C4_BO.xml'

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


No charged group detected, skipping electrostatics.


In [4]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=1, log_write_freq=1, device=cpu, gsd_file_name='PCPDTFBT_C4_BO_10mer_linear_test.gsd',log_file_name="PCPDTFBT_C4_BO_10mer.txt")
sim.run_NVT(n_steps=1, kT=0.1, tau_kt=1.0)
sim.flush_writers()

Initializing simulation state from a gsd.hoomd.Frame.
